### Bloki residualne model 


### Zadanie Implementacyjne: Budowa Modelu Klasyfikacyjnego z Wykorzystaniem Bloków Residualnych na cifar 100
Cel Zadania:
Celem zadania jest zaimplementowanie modelu klasyfikacyjnego, wykorzystującego bloki residualne, które są popularną architekturą w sieciach konwolucyjnych. Model zostanie zbudowany na podstawie zapewnionego kodu, który zawiera definicje bloku residualnego oraz funkcję budującą model.

##### Opis Zadania:
Analiza Kodu: Przeanalizuj dostarczony kod, który zawiera implementację bloku residualnego oraz funkcji budującej model klasyfikacyjny.

* Zwróć uwagę na strukturę bloku residualnego. Jakie operacje są wykonywane w tym bloku?
* Przeanalizuj funkcję build_residual_model. Jakie warstwy są używane w modelu klasyfikacyjnym? Jakie są parametry tych warstw?
##### Implementacja Modelu:

* Wykorzystaj dostarczony kod do zaimplementowania modelu klasyfikacyjnego z wykorzystaniem bloków residualnych.
* W przypadku potrzeby dostosuj parametry modelu, takie jak rozmiar wejściowy czy liczba klas wyjściowych, aby pasowały do Twojego zadania klasyfikacji.
* Przygotowanie Danych:

* Wybierz lub przygotuj zbiór danych do treningu i ewaluacji modelu. Może to być zbiór danych zawierający obrazy należące do różnych klas.
* Przygotuj dane: przeskaluj obrazy do odpowiedniego rozmiaru i przekonwertuj etykiety klas na odpowiedni format (np. one-hot encoding).
##### Trening Modelu:

* Skompiluj model, wybierając odpowiedni optymalizator i funkcję straty. Możesz wykorzystać optymalizator 'adam' i funkcję straty 'categorical_crossentropy', jeśli zadanie klasyfikacji ma więcej niż dwie klasy.
* Przeprowadź trening modelu na przygotowanym zbiorze danych. Ustaw odpowiednią liczbę epok treningu.
##### Ewaluacja Modelu:

Po treningu ocen skuteczność modelu na zestawie testowym. Użyj metryk, takich jak dokładność (accuracy), aby ocenić wydajność modelu.
Zwizualizuj wyniki ewaluacji. 

In [ ]:
from keras.datasets import cifar100
from keras.utils import to_categorical

(X_train, y_train), (X_test, y_test) = cifar100.load_data()

y_train = to_categorical(y_train, num_classes=100)
y_test = to_categorical(y_test, num_classes=100)

In [ ]:
from keras.datasets import cifar100
# example of loading the cifar100 dataset
from matplotlib import pyplot

# load dataset
# summarize loaded dataset
print('Train: X=%s, y=%s' % (X_train.shape, y_train.shape))
print('Test: X=%s, y=%s' % (X_test.shape, y_test.shape))
# plot first few images
for i in range(9):
	# define subplot
	pyplot.subplot(330 + 1 + i)
	# plot raw pixel data
	pyplot.imshow(X_train[i])
# show the figure
pyplot.show()

In [ ]:
# Normalizuj dane wejściowe
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

In [ ]:
# todo model 

def residual_block(input_tensor, filters, kernel_size):
   """
   Tworzy prosty blok residualny.

   Parametry:
   input_tensor - tensor wejściowy do bloku.
   filters - liczba filtrów w warstwach konwolucyjnych.
   kernel_size - rozmiar jądra w warstwach konwolucyjnych.
  
   Zwraca:
   Tensor wyjściowy z bloku residualnego.
   """
   x = layers.Conv2D(filters, kernel_size, padding='same')(input_tensor)
   x = layers.BatchNormalization()(x)
   x = layers.Activation('relu')(x)

   x = layers.Conv2D(filters, kernel_size, padding='same')(x)
   x = layers.BatchNormalization()(x)

   x = layers.add([x, input_tensor])
   x = layers.Activation('relu')(x)

   return x

def build_residual_model(input_shape, num_classes):
   inputs = layers.Input(shape=input_shape)
   # Wstępna warstwa konwolucyjna
   x = layers.Conv2D(64, 7, strides=2, padding='same')(inputs)
   x = layers.BatchNormalization()(x)
   x = layers.Activation('relu')(x)
   x = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)
   # Bloki residualne
   x = residual_block(x, filters=64, kernel_size=3)
   x = residual_block(x, filters=64, kernel_size=3)
   x = residual_block(x, filters=64, kernel_size=3)
   # Warstwa kończąca do klasyfikacji
   x = layers.GlobalAveragePooling2D()(x)
   outputs = layers.Dense(num_classes, activation='softmax')(x)

   model = models.Model(inputs=inputs, outputs=outputs)
   model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

   return model



In [ ]:
y_train.shape

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Przykładowe parametry
input_shape = (32, 32, 3)
num_classes = 100  # Zmień na odpowiednią liczbę klas dla swojego przypadku

# Budowanie i podsumowanie modelu
model = build_residual_model(input_shape, num_classes)
model.summary()


In [ ]:

model.fit(X_train,y_train,verbose=1,batch_size=32, validation_split=0.2, epochs=10)

In [ ]:
model.fit(X_train,y_train,verbose=1,batch_size=64, validation_split=0.2, epochs=50, shuffle=True)

# Few shot learning 

### Zadanie Implementacyjne: Klasyfikacja Obrazów z Wykorzystaniem Few-Shot Learning i ResNet50


Celem zadania jest opracowanie modelu klasyfikacji obrazów z użyciem techniki few-shot learning przy wykorzystaniu wstępnie wytrenowanego modelu ResNet50. Zadanie skupia się na wyzwaniu klasyfikacji, gdy dostępna jest bardzo ograniczona liczba przykładów dla każdej z klasyfikowanych kategorii.



### Przygotowanie Danych:

1. Wybierz zbiór danych do klasyfikacji. Może to być dowolny zbiór danych zawierający obrazy należące do różnych klas. Przykładowo, można wybrać zbiór danych zawierający zdjęcia zwierząt, pojazdów czy obiekty naturalne.
Przygotuj dane: przeskaluj obrazy do rozmiaru 224x224 pikseli, przekonwertuj etykiety klas na format one-hot encoding, podziel zbiór danych na zestawy treningowy, walidacyjny i testowy.
### Modyfikacja i Implementacja Modelu:

2. Wykorzystaj wstępnie wytrenowany model ResNet50, jak pokazano w dostarczonym kodzie. 
* Model ten powinien zostać załadowany bez górnych warstw, aby umożliwić dodanie nowych warstw klasyfikacyjnych.
* Zablokuj trening wszystkich wcześniejszych warstw ResNet50, aby zapobiec ich modyfikacji podczas pierwszego etapu treningu.
* Dodaj warstwy klasyfikacyjne na końcu modelu. 
W przykładzie zastosowano GlobalAveragePooling2D i warstwę Dense z aktywacją softmax, ale zachęcamy do eksperymentowania z różnymi konfiguracjami.
### Trening Modelu:

3. Skompiluj model, używając optymalizatora Adam. 
* Jako funkcję straty wybierz 'categorical_crossentropy', jeśli Twoje zadanie klasyfikacji ma więcej niż dwie klasy.
* Przeprowadź trening modelu na przygotowanym zbiorze danych. Ustaw odpowiednią liczbę epok (w przykładzie jest tylko 1 dla demonstracji, jednak w praktycznych zastosowaniach warto użyć więcej).
* Monitoruj wyniki treningu i walidacji, aby dostosować parametry treningu i uniknąć przetrenowania.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Dense, GlobalAveragePooling2D, AveragePooling2D, Flatten

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
# Definicja generatora obrazów
datagen = ImageDataGenerator(
    rescale=1.0/255,           # Przeskalowanie wartości pikseli do zakresu [0, 1]

)

# Wczytanie danych treningowych
train_generator = datagen.flow_from_directory(
    '/kaggle/input/urban-and-rural-photos/train',
    target_size=(224, 224),    # Rozmiar docelowy obrazów
    batch_size=12,             # Rozmiar wsadu
    class_mode='binary',  # Rodzaj klasyfikacji (binary lub categorical)
    subset='training'          # Ustawienie dla danych treningowych
)


# Wczytanie danych treningowych
test_generator = datagen.flow_from_directory(
    '/kaggle/input/urban-and-rural-photos/val',
    target_size=(224, 224),    # Rozmiar docelowy obrazów
    batch_size=20,             # Rozmiar wsadu
    class_mode='binary',  # Rodzaj klasyfikacji (binary lub categorical)
    subset='validation'          # Ustawienie dla danych treningowych
)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Parametry
image_size = 224  # Zakładając, że używamy tego rozmiaru obrazu
num_classes = 2  # Dla przykładu, zakładamy że mamy 2 klasy: miejskie i wiejskie


# Utworzenie generatora danych bez augmentacji dla zbioru walidacyjnego
validation_data_generator = ImageDataGenerator(rescale=1./255)  # Tylko normalizacja pikseli

# Konfiguracja generatora treningowego z augmentacją
train_data_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_generator = train_data_generator.flow_from_directory(
    '../input/urban-and-rural-photos/train',
    target_size=(image_size, image_size),
    batch_size=12,
    class_mode='categorical')

# Konfiguracja generatora walidacyjnego bez augmentacji
validation_generator = validation_data_generator.flow_from_directory(
    '../input/urban-and-rural-photos/val',
    target_size=(image_size, image_size),
    batch_size=20,
    class_mode='categorical')

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense,Dropout
from tensorflow.keras.optimizers import Adam
output_logistic_regression = 2
from tensorflow.keras.applications import ResNet152

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

# Adding new classification layers
model = Sequential([   base_model,   GlobalAveragePooling2D(),   Dense(output_logistic_regression, activation='softmax')])
model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_generator, epochs=19, verbose=1, validation_data=validation_generator)

In [ ]:
model.fit(train_generator, epochs=10, verbose=1, validation_data=validation_generator)


In [ ]:
model.fit(train_generator, epochs=3, verbose=1, validation_data=validation_generator)

## Segmentation 

### Zadanie: Implementacyjne: U-Net dla Segmentacji Maski Płuc

Celem zadania jest implementacja i trening modelu U-Net na zbiorze danych zawierającym obrazy RTG klatki piersiowej (chest X-ray) wraz z maskami płuc. Zadanie skupia się na praktycznym zastosowaniu modelu U-Net do segmentacji obrazów medycznych, co jest kluczowym zadaniem w analizie medycznej obrazów diagnostycznych.



Celem zadania jest implementacja i trening modelu U-Net na zbiorze danych zawierającym obrazy RTG klatki piersiowej (chest X-ray) wraz z maskami płuc. Zadanie skupia się na praktycznym zastosowaniu modelu U-Net do segmentacji obrazów medycznych, co jest kluczowym zadaniem w analizie medycznej obrazów diagnostycznych.

Opis Zadania:
1. Przygotowanie Danych:
    * Wczytaj dostępny zbiór danych zawierający obrazy RTG klatki piersiowej wraz z odpowiadającymi im maskami płuc. 
    * Przygotuj dane do treningu: przeskaluj obrazy i maski do rozmiaru 128x128 pikseli (lub innego, jeśli uznasz to za stosowne), przekonwertuj maski do binarnej formy, gdzie piksele należące do płuc mają wartość 1, a tło 0.
    * Podziel zbiór danych na zestawy: treningowy, walidacyjny i testowy.
2. Implementacja Modelu U-Net:

    * Wykorzystaj dostarczony kod modelu U-Net. Dokonaj ewentualnych modyfikacji, jeśli jest to konieczne, aby dostosować model do Twoich potrzeb.
    * Zaimplementuj model w środowisku programistycznym wspierającym TensorFlow/Keras.
3. Trening Modelu:
    * Skompiluj model, używając optymalizatora 'adam' i funkcji straty 'binary_crossentropy'. Jako metrykę wybierz 'accuracy'.
4. Ewaluacja Modelu:
    * Oceń skuteczność modelu na zbiorze testowym, wykorzystując odpowiednie metryki, takie jak dokładność.
    * Zwizualizuj wyniki segmentacji na wybranych obrazach z zestawu testowego, porównując maski przewidywane przez model z maskami rzeczywistymi.


In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Ścieżki do folderów z danymi
train_image_dir = '/kaggle/input/lung-mask-image-dataset/ChestXray/train/image'
train_mask_dir = '/kaggle/input/lung-mask-image-dataset/ChestXray/train/mask'
test_image_dir = '/kaggle/input/lung-mask-image-dataset/ChestXray/test/image'
test_mask_dir = '/kaggle/input/lung-mask-image-dataset/ChestXray/test/mask'


image_size = (128, 128)  # Rozmiar do którego będą skalowane obrazy i maski
batch_size = 16  # Rozmiar partii danych

In [ ]:
import tensorflow as tf
import os 

def process_path(image_path, mask_path):
    # Wczytanie i przetworzenie obrazu
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, image_size)
    image = image / 255.0  # Normalizacja do zakresu [0, 1]

    # Wczytanie i przetworzenie maski
    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)
    mask = tf.image.resize(mask, image_size)
    mask = mask / 255.0  # Normalizacja do zakresu [0, 1]

    return image, mask

# Utworzenie listy ścieżek do obrazów i masek
image_paths = [os.path.join(train_image_dir, fname) for fname in os.listdir(train_image_dir)]
mask_paths = [os.path.join(train_mask_dir, fname) for fname in os.listdir(train_mask_dir)]

# Utworzenie tf.data.Dataset
dataset = tf.data.Dataset.from_tensor_slices((image_paths, mask_paths))
dataset = dataset.map(process_path, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.batch(batch_size)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


In [ ]:
dataset

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def conv_block(input_tensor, num_filters):
   encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
   encoder = layers.BatchNormalization()(encoder)
   encoder = layers.Activation('relu')(encoder)
   encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
   encoder = layers.BatchNormalization()(encoder)
   encoder = layers.Activation('relu')(encoder)
   return encoder

def encoder_block(input_tensor, num_filters):
   encoder = conv_block(input_tensor, num_filters)
   encoder_pool = layers.MaxPooling2D((2, 2), strides=(2, 2))(encoder)
   return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
   decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
   decoder = layers.concatenate([concat_tensor, decoder], axis=-1)
   decoder = layers.BatchNormalization()(decoder)
   decoder = layers.Activation('relu')(decoder)
   decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
   decoder = layers.BatchNormalization()(decoder)
   decoder = layers.Activation('relu')(decoder)
   decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
   decoder = layers.BatchNormalization()(decoder)
   decoder = layers.Activation('relu')(decoder)
   return decoder


def build_unet(input_shape):
   inputs = layers.Input(shape=input_shape)

   # Encoder
   encoder0_pool, encoder0 = encoder_block(inputs, 32)
   encoder1_pool, encoder1 = encoder_block(encoder0_pool, 64)
   encoder2_pool, encoder2 = encoder_block(encoder1_pool, 128)
   encoder3_pool, encoder3 = encoder_block(encoder2_pool, 256)

   # Center
   center = conv_block(encoder3_pool, 512)

   # Decoder
   decoder3 = decoder_block(center, encoder3, 256)
   decoder2 = decoder_block(decoder3, encoder2, 128)
   decoder1 = decoder_block(decoder2, encoder1, 64)
   decoder0 = decoder_block(decoder1, encoder0, 32)

   # Output
   outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(decoder0)

   model = models.Model(inputs=[inputs], outputs=[outputs])
   return model


In [ ]:
model=build_unet(input_shape=(128, 128, 3))
model.compile(optimizer=Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(dataset, epochs=10, verbose=1)

In [ ]:
image_paths = [os.path.join(test_image_dir, fname) for fname in os.listdir(test_image_dir)]
mask_paths = [os.path.join(test_mask_dir, fname) for fname in os.listdir(test_mask_dir)]

# Utworzenie tf.data.Dataset
dataset_test = tf.data.Dataset.from_tensor_slices((image_paths, mask_paths))
dataset_test = dataset_test.map(process_path, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset_test = dataset_test.batch(batch_size)
dataset_test = dataset_test.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)




In [ ]:
decoded_images = model.predict(dataset_test)

In [ ]:
import matplotlib.pyplot as plt

n = 20 # Wybierz liczbę przykładów do wyświetlenia
for i in range(n):
    plt.imshow(decoded_images[i],cmap='binary')
    plt.title("Rekonstrukcja")
    plt.show()


In [ ]:
decoded_images[0]-decoded_images[1]